In [290]:
from bs4 import BeautifulSoup
import requests
import datetime
from datetime import date 
from datetime import datetime
import pandas as pd 
import re 
import numpy as np
import sys
import warnings
from datetime import timedelta

if not sys.warnoptions:
    warnings.simplefilter("ignore")

def scrape_odds(link):
    try:
        URL= link
        hdr = {'User-Agent': 'Mozilla/5.0'}
        page= requests.get(URL, headers= hdr)
        maps= page.text 
        soup= BeautifulSoup(maps, 'lxml')

# Bookmaker Names
        names= []
        for element in soup.find_all('a',{"class": "bk-logo-main-90 bk-logo-click"}):
            names.append(element['title'])
    
        names.insert(0, 'Outcome')

# The odds and Team Names 
        odds= []
        teams= []

        soup_odds= soup.find('tbody')

        for row in soup_odds.find_all('tr'):
            cells= row.find_all('td')
            outcome= cells[0].find('span').find('span')['data-name']
            opus= [x['data-odig'] for x in cells if x.has_attr('data-odig')]
            opus.insert(0, outcome)
            odds.append(opus)
            teams.append(outcome)

        date= soup.find('div',{'class':'event'}).text
        fixture= soup.find('h1').text
        scrape_time= pd.to_datetime(datetime.now(), format="%d/%m/%Y %H:%M:%S")

        data= pd.DataFrame(odds, columns= names)

        p= re.compile('^.+?(?=\ v)')

        home_team= p.search(fixture).group(0)#[:-1]

        lop= list(data['Outcome'])

        if home_team== lop[0] and lop[1] != 'Draw':
            order= [1,2,0]
            lop= [lop[i] for i in order]

        if home_team== lop[0] and lop[1] == 'Draw':
            order= [2,1,0]
            lop= [lop[i] for i in order]
            
        if home_team== lop[1] and lop[2] == 'Draw':
            order= [0,2,1]
            lop= [lop[i] for i in order]
            
        if home_team== lop[1] and lop[2] != 'Draw':
            order= [2,0,1]
            lop= [lop[i] for i in order]
            
        if home_team== lop[2] and lop[1] == 'Draw':
            order= [0,1,2]
            lop= [lop[i] for i in order]
            
        if home_team== lop[2] and lop[1] != 'Draw':
            order= [1,0,2]
            lop= [lop[i] for i in order]
    
        data['Outcome']= pd.Categorical(data['Outcome'], lop)
        data= data.sort_values('Outcome').reset_index(drop= True)

        a_list= list(data.columns[1:])
        b_list= ['H','D','A']

        oo= ['{}{}'.format(a,b) for b in b_list for a in a_list]

        oo.sort()

        for i in np.arange(0,len(oo), 3):
            home= oo[i+2]
            away= oo[i]
            oo[i]= home
            oo[i+2]= away
 
        oo.insert(0, 'scrape_time')
        oo.insert(0, 'date')
        oo.insert(0, 'away_team')
        oo.insert(0,'home_team')

        new= pd.DataFrame(columns= oo, index= np.arange(0,1))

        new['home_team']= data['Outcome'][2]
        new['away_team']= data['Outcome'][0]

        date= soup.find('div',{'class':'event'}).text.split(',',1)[1][1:].\
        lower().replace("rd", "").replace("nd", "").replace("st", "").replace("th", "")

        num= [i for i,x in enumerate(date) if x.isspace()][1]

        date= date[:num]+ ' '+ str(scrape_time.year) + date[num:]

        date= pd.to_datetime(datetime.strptime(date, '%d %B %Y %H:%M'),format= "%Y-%m-d %H:%M")
        hours_added = timedelta(hours = 1)
        new['date']= date+hours_added
        new['scrape_time']= scrape_time

        for name, stuff in data.drop('Outcome',axis=1).iteritems():
            for snam, snugg in new.iteritems():
                if name in snam and snam.endswith('A'):
                    snugg[0]= stuff[0]
                if name in snam and snam.endswith('D'):
                    snugg[0]= stuff[1]         
                if name in snam and snam.endswith('H'):
                    snugg[0]= stuff[2]
            
        new[new.columns.difference(['home_team', 'away_team','date','scrape_time'])]= new[new.columns.difference(['home_team', 'away_team','date','scrape_time'])].astype('float')
        return(new)
    
    except:
        new= pd.DataFrame()
        return(new)


In [135]:
def get_leagues(link):
    lan= open('länder.txt','r').readlines()
    land= [x[:-1] for x in lan]
    URL= link
    hdr = {'User-Agent': 'Mozilla/5.0'}
    page= requests.get(URL, headers= hdr)
    maps= page.text 
    soup= BeautifulSoup(maps, 'lxml')
    links= []
    for element in soup.find_all('loc'):
        s= element.text
        if s.count('/') < 6 and any(thing in s for thing in lander) and not s.endswith('winner'):
            links.append(s)
    return(links)

In [278]:
errors= []
results= pd.DataFrame()
#links= get_leagues('https://www.oddschecker.com/sport/football/sitemap.xml')
for thing in links:
    hdr = {'User-Agent': 'Mozilla/5.0'}
    page= requests.get(thing, headers= hdr)
    maps= page.text 
    soup= BeautifulSoup(maps, 'lxml')
    league= thing[thing.rindex('/')+1:]
    for element in soup.find_all('a',{'class':'beta-callout full-height-link whole-row-link'}):
        try:
            new= scrape_odds('https://www.oddschecker.com'+element['href'])
            new['league']= league
            results= results.append(new)
        except:
            errors.append(element) 

In [289]:
#results= results.drop_duplicates(subset=['home_team','away_team'])
#results.head()

#hours_added

datetime.timedelta(seconds=3600)

In [233]:
#results.sort_values('date').head(100
results.sort_values('date').head(30)
results.query('home_team== "Draw"')

,home_team,away_team,date,scrape_time,10BetH,10BetD,10BetA,888sportH,888sportD,888sportA,...,VBetH,VBetD,VBetA,William HillH,William HillD,William HillA,league,ToteH,ToteD,ToteA
0,Draw,Brighton,2021-05-09 12:00:00,2021-04-30 13:02:02.294674,3.30,2.70,2.65,3.10,2.75,0.00,...,3.35,2.71,2.72,3.10,2.80,2.70,premier-league,NaN,NaN,NaN
0,Draw,Crewe,2021-05-01 15:00:00,2021-04-30 13:02:35.325477,3.70,2.75,2.40,3.55,2.75,2.40,...,3.74,2.79,2.39,3.40,2.80,2.45,league-1,NaN,NaN,NaN
0,Draw,Stevenage,2021-05-01 15:00:00,2021-04-30 13:03:05.183692,3.25,3.20,2.30,3.25,3.20,2.25,...,3.26,3.20,2.27,3.25,3.20,2.30,league-2,NaN,NaN,NaN
0,Draw,Aberdeen,2021-05-01 15:00:00,2021-04-30 13:03:17.993496,3.20,2.80,2.60,3.10,2.75,2.65,...,3.23,2.77,2.68,3.10,2.80,2.62,premiership,NaN,NaN,NaN
0,Draw,Cagliari,2021-05-09 14:00:00,2021-04-30 13:05:56.031969,3.35,3.00,2.15,3.38,3.05,2.20,...,3.53,3.21,2.27,3.40,3.00,2.30,serie-a,NaN,NaN,NaN
0,Draw,Benfica B,2021-05-02 11:15:00,2021-04-30 13:10:56.841748,3.25,3.00,2.30,3.10,2.95,2.25,...,0.00,0.00,0.00,3.20,3.10,2.30,segunda-liga,NaN,NaN,NaN
0,Draw,FC Minsk Reserves,2021-04-30 15:00:00,2021-04-30 13:11:50.270741,4.30,2.80,1.95,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,belarus-reserve-league,NaN,NaN,NaN
0,Draw,Sparta Prague,2021-05-02 16:00:00,2021-04-30 13:12:43.291523,3.45,2.55,2.50,3.40,2.60,2.45,...,3.52,2.61,2.52,3.40,2.70,2.50,synot-liga,NaN,NaN,NaN
0,Draw,Jihlava,2021-04-30 16:00:00,2021-04-30 13:12:51.936598,3.10,3.10,2.25,3.15,0.00,2.20,...,0.00,0.00,0.00,3.10,3.10,2.25,druha-liga,NaN,NaN,NaN
0,Draw,FA 2000,2021-05-02 12:00:00,2021-04-30 13:14:07.552461,3.20,2.65,2.40,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,2nd-division,NaN,NaN,NaN


In [291]:
scrape_odds('https://www.oddschecker.com/football/israel/premier-league/maccabi-petach-tikva-v-maccabi-haifa/winner')

,home_team,away_team,date,scrape_time,10BetH,10BetD,10BetA,888sportH,888sportD,888sportA,...,SpreadexA,UnibetH,UnibetD,UnibetA,VBetH,VBetD,VBetA,William HillH,William HillD,William HillA
0,Maccabi Petach Tikva,Maccabi Haifa,2021-05-01 19:30:00,2021-04-30 19:27:49.880006,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [183]:
#TO DO 
#2. Find reason for duplicates 
#3. Include Pinnacle Odds 
#4. Find a way to merge pinnacle odds into the results frame 
#5. Calculate Margins 

Index(['home_team', 'away_team', 'date', 'scrape_time', '10BetH', '10BetD',
       '10BetA', '888sportH', '888sportD', '888sportA', 'Bet VictorH',
       'Bet VictorD', 'Bet VictorA', 'Bet365H', 'Bet365D', 'Bet365A',
       'Betfair SportsbookH', 'Betfair SportsbookD', 'Betfair SportsbookA',
       'BetfairH', 'BetfairD', 'BetfairA', 'BetfredH', 'BetfredD', 'BetfredA',
       'BetwayH', 'BetwayD', 'BetwayA', 'Boyle SportsH', 'Boyle SportsD',
       'Boyle SportsA', 'GentingBetH', 'GentingBetD', 'GentingBetA',
       'Mansion BetH', 'Mansion BetD', 'Mansion BetA', 'MatchbookH',
       'MatchbookD', 'MatchbookA', 'NovibetH', 'NovibetD', 'NovibetA',
       'Paddy PowerH', 'Paddy PowerD', 'Paddy PowerA', 'RedzoneH', 'RedzoneD',
       'RedzoneA', 'SkybetH', 'SkybetD', 'SkybetA', 'Smarkets SportsbookH',
       'Smarkets SportsbookD', 'Smarkets SportsbookA', 'SmarketsH',
       'SmarketsD', 'SmarketsA', 'Sport NationH', 'Sport NationD',
       'Sport NationA', 'Sporting IndexH', 'Sporting In

In [281]:
results

,home_team,away_team,date,scrape_time,10BetH,10BetD,10BetA,888sportH,888sportD,888sportA,...,UnibetH,UnibetD,UnibetA,VBetH,VBetD,VBetA,William HillH,William HillD,William HillA,league
0,Man City,PSG,2021-05-04 20:00:00,2021-04-30 19:21:01.774778,1.66,4.05,4.60,1.67,4.10,4.40,...,1.75,4.05,4.50,1.69,4.20,4.75,1.70,4.20,4.60,champions-league
0,Chelsea,Real Madrid,2021-05-05 20:00:00,2021-04-30 19:21:02.710391,2.20,3.35,3.40,2.15,3.25,3.30,...,2.25,3.40,3.30,2.21,3.35,3.45,2.15,3.25,3.60,champions-league
0,Arsenal,Villarreal,2021-05-06 20:00:00,2021-04-30 19:21:05.523674,1.83,3.35,4.35,1.85,3.45,4.10,...,1.90,3.50,4.25,1.90,3.45,4.50,1.85,3.40,4.40,europa-league
0,Roma,Man Utd,2021-05-06 20:00:00,2021-04-30 19:21:09.006705,3.70,3.90,1.86,3.70,3.85,1.85,...,3.85,3.90,1.90,3.85,4.00,1.88,3.70,3.90,1.91,europa-league
